In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import csv
import gmaps

# Google API Key
from config import gkey

# Configure gmaps with API key
gmaps.configure(api_key=gkey)

%matplotlib inline

In [2]:
# Import CSV
gy_df = pd.read_csv("Resources/yelp_output.csv", error_bad_lines=False, encoding="UTF-16LE")

gy_df = gy_df.rename(columns={"yelp_rating": "Yelp Rating",
                              "yelp_review_count": "Yelp Review Count"
                             })

gy_df_clean = gy_df[["Restaurant Name", "Street Address", "Lat", "Long", "Inspection Date",
                     "Inspection Score", "Rating", "Yelp Rating", "Rating Count", "Yelp Review Count"
                    ]]

gy_df_clean = gy_df_clean.rename(columns={"Rating": "Google Rating",
                                         "Rating Count": "Google Rating Count"})
gy_df_clean

gy_df_drop = gy_df_clean.copy()

gy_df_drop = gy_df_drop[pd.notnull(gy_df_drop['Yelp Rating'])]

gy_df_drop

,Restaurant Name,Street Address,Lat,Long,Inspection Date,Inspection Score,Google Rating,Yelp Rating,Google Rating Count,Yelp Review Count
1,A & J BAKERY,3515 OAK LAWN AVE,32.812380,-96.805424,4/30/2019,92,4.0,3.0,60,55.0
3,A PLUS GROCERY BEER & WINE,4407 W MOCKINGBIRD LN,32.836885,-96.830941,2/14/2019,82,4.3,3.0,44,29.0
4,A SANDWICH SHOP,8333 DOUGLAS AVE #C140,32.864038,-96.809478,1/28/2019,92,5.0,3.0,1,2.0
5,A TASTE OF WAYS CATERING,2949 E KIEST BLVD #A,32.726334,-96.786419,12/11/2019,97,4.6,4.5,97,6.0
6,A TO Z FOOD MART,2120 N ST AUGUSTINE RD #130,32.749386,-96.656170,6/17/2019,97,4.4,2.0,96,1.0
...,...,...,...,...,...,...,...,...,...,...
2164,ZIO CECIO,4615 W LOVERS LN,32.851635,-96.830285,7/23/2019,97,4.5,4.0,110,108.0
2165,ZIZI RAMEN AND SUSHI,9711 PLANO RD STE #B,32.879653,-96.701368,8/23/2019,89,4.6,4.5,118,76.0
2166,ZIZIKI'S,4514 TRAVIS ST #122,32.822316,-96.790437,12/16/2019,80,4.6,4.0,332,353.0
2167,ZOE'S KITCHEN,3878 OAK LAWN #314,32.815485,-96.800690,11/26/2019,90,4.2,3.0,334,79.0


In [ ]:
narrowed_restaurant = gy_df_drop.copy()

# User chosen conditions
inspectionScore = int(input("What inspection score would you ideally like to eat at? "))
googleRating = int(input("From 1-5, what's the minimum Google rating you're looking to eat at? "))
yelpRating = int(input("From 1-5, what's the minimum Yelp rating you're looking to eat at? "))

# Apply filters
narrowed_restaurant = narrowed_restaurant[narrowed_restaurant['Inspection Score'] >= inspectionScore]
narrowed_restaurant = narrowed_restaurant[narrowed_restaurant['Google Rating'] >= googleRating]
narrowed_restaurant = narrowed_restaurant[narrowed_restaurant['Yelp Rating'] >= yelpRating]  

# Store 'Lat' and 'Lng' into  locations 
locations = narrowed_restaurant[["Lat", "Long"]]
rating = narrowed_restaurant["Inspection Score"].astype(int)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,weights=rating, 
                                 dissipating=True, max_intensity=100,
                                 point_radius = 40)

# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Restaurant</dt><dd>{Restaurant Name}</dd>
<dt>Address</dt><dd>{Street Address}</dd>
<dt>Google Rating</dt><dd>{Google Rating}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
restaurant_info = [info_box_template.format(**row) for index, row in narrowed_restaurant.iterrows()]
locations = narrowed_restaurant[["Lat", "Long"]]

# Add marker layer ontop of heat map

# Optional: Mouseover
info_box_template2 = '''
Restaurant: {Restaurant Name}
Address: {Street Address}
Google Rating: {Google Rating}

'''

restaurant_info2 = [info_box_template2.format(**row) for index, row in narrowed_restaurant.iterrows()]

markers = gmaps.marker_layer(locations, info_box_content=restaurant_info, hover_text=restaurant_info2)


# Display Map
fig.add_layer(heat_layer)
fig.add_layer(markers)

fig